# Download ECMWF forecast data from MS Planetary Computer
- https://planetarycomputer.microsoft.com/dataset/ecmwf-forecast#overview
- parameter database: https://apps.ecmwf.int/codes/grib/param-db/

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import urllib.request
import tempfile

import httpx
import planetary_computer
import pystac_client
from tqdm import tqdm
import xarray as xr

from download import download_file

## Query STAC to get data URLs

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [ ]:
forecast_step = "0h"

In [ ]:
search = catalog.search(
    collections=["ecmwf-forecast"],
    query={
        "ecmwf:stream": {"eq": "scda"},
        "ecmwf:step": {"eq": forecast_step},
    },
)
items = search.get_all_items()
len(items)

In [ ]:
item = max(items, key=lambda item: item.datetime)
item

In [ ]:
item.properties

In [ ]:
data_url = item.assets["data"].href
index_url = item.assets["index"].href

## Use Range to download only part of the file we want

In [ ]:
# Get byte off
r = httpx.get(index_url)
assert r.status_code == 200
chunks = [json.loads(t) for t in r.text.strip().split("\n")]
ch = [c for c in chunks if c["param"] == "t"][0]
print(ch)

In [ ]:
set(c["param"] for c in chunks)

In [ ]:
offset, length = ch["_offset"], ch["_length"]
start, end = offset, offset + length
headers = {"Range": f"bytes={start}-{end}"}
# headers = {}

## Download file to /tmp dir

In [ ]:
file = download_file(data_url, headers=headers)

## Open and explore dataset

In [ ]:
ds = xr.open_dataset(
    file,
    engine="cfgrib",
    # filter_by_keys={"dataType": "cf"},
)
ds

In [ ]:
keys = {
    # "u10": "wind u",
    # "v10": "wind v",
    "t": "air temp",
    # downward flux: missing
    # tp values are 0?
    # precip rate missing
}

In [ ]:
ds.sel(
    latitude=slice(45, 40),
    longitude=slice(10, 15),
    # isobaricInhPa=1000,
)[keys.keys()].t